In [1]:
import math
import pathlib

import numpy as np
import tensorflow as tf


In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
	model = tf.keras.Sequential()
	# foundation for 10x30 image
	n_nodes = 256 * 10 * 30
	model.add(tf.keras.layers.Dense(n_nodes, input_dim=latent_dim))
	model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
	model.add(tf.keras.layers.Reshape((10, 30, 256)))
	# upsample to 20x60
	model.add(tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
	# upsample to 40x120
	model.add(tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
	# upsample to 80x240
	model.add(tf.keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
	# output layer
	model.add(tf.keras.layers.Conv2D(3, (3,3), activation='tanh', padding='same'))
	return model

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(80,240,3)):
	model = tf.keras.Sequential()
	# normal
	model.add(tf.keras.layers.Conv2D(64, (3,3), padding='same', input_shape=in_shape))
	model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
	# downsample
	model.add(tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
	# downsample
	model.add(tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
	# downsample
	model.add(tf.keras.layers.Conv2D(256, (3,3), strides=(2,2), padding='same'))
	model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
	# classifier
	model.add(tf.keras.layers.Flatten())
	model.add(tf.keras.layers.Dropout(0.4))
	model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
	# compile model
	opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = tf.keras.Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = tf.keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [ ]:
def path_to_real_data(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [80, 240])
    img = (img - 127.5) / 127.5
    return img


def generate_real_samples(begin, end):
    real_images = [path_to_real_data(f) for f in files[begin:end]]
    real_labels = np.ones(len(real_images))
    return real_images, real_labels

files = [str(p) for p in pathlib.Path('data').glob("*")]


In [ ]:
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = np.random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [ ]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = np.zeros((n_samples, 1))
	return X, y

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=800):
    # manually enumerate epochs
    batch_size = 2
    batch = math.ceil(len(files) / batch_size)

    for i in range(n_epochs):
        for b in range(batch):
            # get 'real' samples
            x_real, y_real = generate_real_samples(b * batch_size, (b+1) * batch_size)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(np.array(x_real), y_real)
            # generate 'fake' examples
            x_fake, y_fake = generate_fake_samples(g_model, latent_dim, batch_size)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(x_fake, y_fake)
            # prepare points in latent space as input for the generator
            x_gan = generate_latent_points(latent_dim, batch_size)
            # create inverted labels for the fake samples
            y_gan = np.ones((batch_size, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(x_gan, y_gan)
            # summarize loss on this batch
            print('%d, b=%d, d1=%.3f, d2=%.3f g=%.3f' % (i + 1, b + 1, d_loss1, d_loss2, g_loss))

In [ ]:
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# train model
train(g_model, d_model, gan_model, latent_dim)

In [ ]:
import matplotlib.pyplot  as plt

fakes_x, fake_y = generate_fake_samples(g_model, latent_dim, 1)
fakes_x = ((fakes_x * 127.5) + 127.5) / 255.
plt.imshow(fakes_x[0])
plt.show()